### Set Up

In [3]:
import os
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import seaborn as sns
from pytorch_forecasting import TimeSeriesDataSet

BASE_PATH = 'D:/Data/KIMoDIs/global-groundwater-models-main'
DATA_PATH = os.path.join(BASE_PATH, 'data')
MODEL_PATH = os.path.join(BASE_PATH, 'models')
RESULT_PATH = os.path.join(BASE_PATH, 'results')

LAG = 52  # weeks
LEAD = 12  # weeks

# Roughly 80/10/10
TRAIN_PERIOD = (pd.Timestamp(1990, 1, 1), pd.Timestamp(2010, 1, 1))
VAL_PERIOD = (pd.Timestamp(2010, 1, 1), pd.Timestamp(2013, 1, 1))
TEST_PERIOD = (pd.Timestamp(2013, 1, 1), pd.Timestamp(2016, 1, 1))

TIME_IDX = pd.date_range(TRAIN_PERIOD[0], TEST_PERIOD[1], freq='W-SUN', inclusive='neither', name='time').to_frame().reset_index(drop=True)
TIME_IDX.index.name = 'time_idx'
TIME_IDX = TIME_IDX.reset_index()

In [4]:
train_df = pq.read_table(os.path.join(DATA_PATH, 'train_df.parquet'))
val_df = pq.read_table(os.path.join(DATA_PATH, 'val_df.parquet'))
test_df = pq.read_table(os.path.join(DATA_PATH, 'test_df.parquet'))

train_df = train_df.to_pandas()
val_df = val_df.to_pandas()
test_df = test_df.to_pandas()

In [5]:
# Set training, validation and test data to common ids
id_test = test_df['proj_id'].unique()

train_df = train_df[train_df['proj_id'].isin(id_test)]
val_df = val_df[val_df['proj_id'].isin(id_test)]

## Time Series Data Set

In [6]:
list(train_df)

['proj_id',
 'time',
 'gwl',
 'precip',
 'humid',
 'temp',
 'lai',
 'day_sin',
 'day_cos',
 'twi',
 'gw_recharge',
 'hyraum_gr',
 'aquifer_type',
 'permeability_coef',
 'soil_texture',
 'elevation',
 'land_cover',
 'landform_entr10km',
 'landform_sha10km',
 'landform_uni10km',
 'eumohp_dsd1',
 'eumohp_dsd2',
 'eumohp_dsd3',
 'eumohp_dsd4',
 'eumohp_dsd5',
 'eumohp_lp1',
 'eumohp_lp2',
 'eumohp_lp3',
 'eumohp_lp4',
 'eumohp_lp5',
 'eumohp_sd1',
 'eumohp_sd2',
 'eumohp_sd3',
 'eumohp_sd4',
 'eumohp_sd5',
 'lon',
 'lat',
 'sin_temp',
 'time_idx']

In [7]:
train_df.dtypes

proj_id                      object
time                 datetime64[ns]
gwl                         float32
precip                      float64
humid                       float64
temp                        float64
lai                         float64
day_sin                     float32
day_cos                     float32
twi                         float64
gw_recharge                 float64
hyraum_gr                  category
aquifer_type               category
permeability_coef          category
soil_texture               category
elevation                   float32
land_cover                 category
landform_entr10km           float64
landform_sha10km            float64
landform_uni10km            float64
eumohp_dsd1                 float64
eumohp_dsd2                 float64
eumohp_dsd3                 float64
eumohp_dsd4                 float64
eumohp_dsd5                 float64
eumohp_lp1                  float64
eumohp_lp2                  float64
eumohp_lp3                  

In [11]:
# TFT_full
train_ds_tft = TimeSeriesDataSet(
    train_df,
    group_ids=["proj_id"],
    target="gwl",
    time_idx="time_idx",
    min_encoder_length=LAG,
    max_encoder_length=LAG,
    min_prediction_length=1,
    max_prediction_length=LEAD,
    static_reals=['twi',
                  'gw_recharge',
                  'landform_sha10km',
                  'eumohp_dsd1',
                  'eumohp_lp1',
                  'eumohp_sd1',
                  'elevation'],
    static_categoricals=['hyraum_gr',
                         'land_cover', 
                         'aquifer_type', 
                         'permeability_coef',
                         'soil_texture'],
    time_varying_unknown_reals=['gwl'],
    time_varying_known_reals=['humid',
                              'precip',
                              'temp',
                              'sin_temp',
                              'lai',
                              'day_sin',
                              'day_cos'],
    add_target_scales=True, # Adds center and scale of the unnormalized timeseries as feature to static reals
    allow_missing_timesteps=False
)

val_ds_tft = TimeSeriesDataSet.from_dataset(train_ds_tft, val_df)

train_ds_tft.save(os.path.join(RESULT_PATH, 'preprocessing', 'train_ds_full_tft.pt'))
val_ds_tft.save(os.path.join(RESULT_PATH, 'preprocessing', 'val_ds_full_tft.pt'))
train_ds_tft.save('J:/B22-FISHy/NUTZER/Kunz.S/kimodis_preprocessed/train_ds_full_tft.pt')
val_ds_tft.save('J:/B22-FISHy/NUTZER/Kunz.S/kimodis_preprocessed/val_ds_full_tft.pt')

In [12]:
len(train_ds_tft.decoded_index['proj_id'].unique())

5308

In [13]:
# TFT_interpol (without without GWL as input)
train_ds_tft_interpol = TimeSeriesDataSet(
    train_df,
    group_ids=["proj_id"],
    target="gwl",
    time_idx="time_idx",
    min_encoder_length=LAG,
    max_encoder_length=LAG,
    min_prediction_length=1,
    max_prediction_length=LEAD,
    static_reals=['twi',
                  'gw_recharge',
                  'landform_sha10km',
                  'eumohp_dsd1',
                  'eumohp_lp1',
                  'eumohp_sd1',
                  'elevation'],
    static_categoricals=['hyraum_gr',
                         'land_cover', 
                         'aquifer_type', 
                         'permeability_coef',
                         'soil_texture'],
    time_varying_unknown_reals=[],
    time_varying_known_reals=['humid',
                              'precip',
                              'temp',
                              'sin_temp',
                              'lai',
                              'day_sin',
                              'day_cos'],
    add_target_scales=False, # Do not add center and scale of the unnormalized timeseries as feature to static reals
    allow_missing_timesteps=False
)

val_ds_tft_interpol = TimeSeriesDataSet.from_dataset(train_ds_tft_interpol, val_df)

train_ds_tft_interpol.save(os.path.join(RESULT_PATH, 'preprocessing', 'train_ds_full_interpol_tft.pt'))
val_ds_tft_interpol.save(os.path.join(RESULT_PATH, 'preprocessing', 'val_ds_full_interpol_tft.pt'))
train_ds_tft_interpol.save('J:/B22-FISHy/NUTZER/Kunz.S/kimodis_preprocessed/train_ds_full_interpol_tft.pt')
val_ds_tft_interpol.save('J:/B22-FISHy/NUTZER/Kunz.S/kimodis_preprocessed/val_ds_full_interpol_tft.pt')

In [ ]:
# Checking the TimeSeriesDataSets

# train_ds_tft.get_parameters()
# dataloader = train_ds_tft.to_dataloader(batch_size=4)

# and load the first batch
# x, y = next(iter(dataloader))
# print("x =", x)
# print("\ny =", y)
# print("\nsizes of x =")
# for key, value in x.items():
#     print(f"\t{key} = {value.size()}")
    
# val_ds_tft.get_parameters()
# # convert the dataset to a dataloader
# dataloader = val_ds_tft.to_dataloader(batch_size=4)

# # and load the first batch
# x, y = next(iter(dataloader))
# print("x =", x)
# print("\ny =", y)
# print("\nsizes of x =")
# for key, value in x.items():
#     print(f"\t{key} = {value.size()}")

In [14]:
# TFT_dyn only with dynamic input variables (including the groundwaterlevel)
train_ds_tft_dyn = TimeSeriesDataSet(
    train_df,
    group_ids=["proj_id"],
    target="gwl",
    time_idx="time_idx",
    min_encoder_length=LAG,
    max_encoder_length=LAG,
    min_prediction_length=1,
    max_prediction_length=LEAD,
    static_reals=[],
    static_categoricals=[],
    time_varying_unknown_reals=['gwl'],
    time_varying_known_reals=['humid',
                              'precip',
                              'temp',
                              'sin_temp',
                              'lai',
                              'day_sin',
                              'day_cos'],
    add_target_scales=False, 
    allow_missing_timesteps=False
)

val_ds_tft_dyn = TimeSeriesDataSet.from_dataset(train_ds_tft_dyn, val_df)

train_ds_tft_dyn.save(os.path.join(RESULT_PATH, 'preprocessing', 'train_ds_dyn_tft.pt'))
val_ds_tft_dyn.save(os.path.join(RESULT_PATH, 'preprocessing', 'val_ds_dyn_tft.pt'))
train_ds_tft_dyn.save('J:/B22-FISHy/NUTZER/Kunz.S/kimodis_preprocessed/train_ds_dyn_tft.pt')
val_ds_tft_dyn.save('J:/B22-FISHy/NUTZER/Kunz.S/kimodis_preprocessed/val_ds_dyn_tft.pt')

In [15]:
# N-HiTS_full
# min_prediction_length is set to max_prediction_length
train_ds_nhits = TimeSeriesDataSet(
    train_df,
    group_ids=['proj_id'],
    target='gwl',
    time_idx='time_idx',
    min_encoder_length=LAG,
    max_encoder_length=LAG,
    min_prediction_length=LEAD,
    max_prediction_length=LEAD,
    static_reals=['twi',
                  'gw_recharge',
                  'landform_sha10km',
                  'eumohp_dsd1',
                  'eumohp_lp1',
                  'eumohp_sd1',
                  'elevation'],
    static_categoricals=['hyraum_gr',
                         'land_cover', 
                         'aquifer_type', 
                         'permeability_coef',
                         'soil_texture'],
    time_varying_unknown_reals=['gwl'],
    time_varying_known_reals=['humid',
                              'precip',
                              'temp',
                              'sin_temp',
                              'lai',
                              'day_sin',
                              'day_cos'],
    add_target_scales=True, # Adds center and scale of the unnormalized timeseries as feature to static reals
    allow_missing_timesteps=False
)

val_ds_nhits = TimeSeriesDataSet.from_dataset(train_ds_nhits, val_df)

train_ds_nhits.save(os.path.join(RESULT_PATH, 'preprocessing', 'train_ds_full_nhits.pt'))
val_ds_nhits.save(os.path.join(RESULT_PATH, 'preprocessing', 'val_ds_full_nhits.pt'))
train_ds_nhits.save('J:/B22-FISHy/NUTZER/Kunz.S/kimodis_preprocessed/train_ds_full_nhits.pt')
val_ds_nhits.save('J:/B22-FISHy/NUTZER/Kunz.S/kimodis_preprocessed/val_ds_full_nhits.pt')

In [16]:
# N-HiTS_interpol
# Interstingly, this model variant doesn't work, i.e. N-HiTS does need the target variable as input  
# Would be interesting to understand why
train_ds_nhits_interpol = TimeSeriesDataSet(
    train_df,
    group_ids=['proj_id'],
    target='gwl',
    time_idx='time_idx',
    min_encoder_length=LAG,
    max_encoder_length=LAG,
    min_prediction_length=LEAD,
    max_prediction_length=LEAD,
    static_reals=['twi',
                  'gw_recharge',
                  'landform_sha10km',
                  'eumohp_dsd1',
                  'eumohp_lp1',
                  'eumohp_sd1',
                  'elevation'],
    static_categoricals=['hyraum_gr',
                         'land_cover', 
                         'aquifer_type', 
                         'permeability_coef',
                         'soil_texture'],
    time_varying_unknown_reals=[],
    time_varying_known_reals=['humid',
                              'precip',
                              'temp',
                              'sin_temp',
                              'lai',
                              'day_sin',
                              'day_cos'],
    add_target_scales=False, # Adds center and scale of the unnormalized timeseries as feature to static reals
    allow_missing_timesteps=False
)

val_ds_nhits_interpol = TimeSeriesDataSet.from_dataset(train_ds_nhits_interpol, val_df)

train_ds_nhits_interpol.save(os.path.join(RESULT_PATH, 'preprocessing', 'train_ds_full_interpol_nhits.pt'))
val_ds_nhits_interpol.save(os.path.join(RESULT_PATH, 'preprocessing', 'val_ds_full_interpol_nhits.pt'))
train_ds_nhits_interpol.save('J:/B22-FISHy/NUTZER/Kunz.S/kimodis_preprocessed/train_ds_full_interpol_nhits.pt')
val_ds_nhits_interpol.save('J:/B22-FISHy/NUTZER/Kunz.S/kimodis_preprocessed/val_ds_full_interpol_nhits.pt')

In [17]:
# N-HiTS_dyn only with dynamic input (with GWL)
train_ds_nhits_dyn = TimeSeriesDataSet(
    train_df,
    group_ids=["proj_id"],
    target="gwl",
    time_idx="time_idx",
    min_encoder_length=LAG,
    max_encoder_length=LAG,
    min_prediction_length=LEAD,
    max_prediction_length=LEAD,
    static_reals=[],
    static_categoricals=[],
    time_varying_unknown_reals=['gwl'],
    time_varying_known_reals=['humid',
                              'precip',
                              'temp',
                              'sin_temp',
                              'lai',
                              'day_sin',
                              'day_cos'],
    add_target_scales=False, # Adds center and scale of the unnormalized timeseries as feature to static reals
    allow_missing_timesteps=False
)

val_ds_nhits_dyn = TimeSeriesDataSet.from_dataset(train_ds_nhits_dyn, val_df)

train_ds_nhits_dyn.save(os.path.join(RESULT_PATH, 'preprocessing', 'train_ds_dyn_nhits.pt'))
val_ds_nhits_dyn.save(os.path.join(RESULT_PATH, 'preprocessing', 'val_ds_dyn_nhits.pt'))
train_ds_nhits_dyn.save('J:/B22-FISHy/NUTZER/Kunz.S/kimodis_preprocessed/train_ds_dyn_nhits.pt')
val_ds_nhits_dyn.save('J:/B22-FISHy/NUTZER/Kunz.S/kimodis_preprocessed/val_ds_dyn_nhits.pt')